# Data Imputation and Feature Engineering

In [1]:
# !pip install tmdbv3api

In [1]:
API_KEY = '5b9105a64cdd16b8cc9259f36cae74d0'

In [2]:
from tmdbv3api import Movie

In [3]:
from tmdbv3api import TMDb

In [4]:
import requests

In [5]:
tmdb = TMDb()
tmdb.api_key = API_KEY

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
from collections import Counter

In [8]:
import os 
import sys

In [9]:
from tqdm import tqdm

In [10]:
import json

In [11]:
import tqdm.notebook as tq

In [12]:
from pandas import Panel

/home/tanmay/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  """Entry point for launching an IPython kernel.


In [13]:
tqdm.pandas()

/home/tanmay/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [14]:
from pandarallel import pandarallel

In [15]:
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


## Utils

In [10]:
def split_names(x):
    
    """
    A df function that will split a string of names for further processing
    """
    
    return [n for n in x.split(',') if (not pd.isna(x) and not x == '')]

## Data Imputation

The first step is to impute missing data from our datasets.
Feature to be imputed are:
- Genres
- runtime
- num votes


### Genres

In [11]:
title_rating = pd.read_csv('processed/title_rating.csv')

In [12]:
title_rating.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000502,movie,Bohemios,Bohemios,0,1970-01-01 00:00:00.000001905,NaN,100.0,NaN,4.5,14
1,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1970-01-01 00:00:00.000001906,NaN,70.0,"Action,Adventure,Biography",6.0,754
2,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1970-01-01 00:00:00.000001907,NaN,90.0,Drama,4.6,17
3,tt0000615,movie,Robbery Under Arms,Robbery Under Arms,0,1970-01-01 00:00:00.000001907,NaN,NaN,Drama,4.5,23
4,tt0000630,movie,Hamlet,Amleto,0,1970-01-01 00:00:00.000001908,NaN,NaN,Drama,3.8,24


In [13]:
title_rating.genres.fillna('', inplace = True)

In [14]:
title_rating.genres.isna().sum()

0

We have 11,666 values that need to be imputed

**Imputation strategy**

One way to impute genre values is to consider the people working on that movie. Generally people tend to work on similar movies and we can use this hypothesis to impute genre values 

In [15]:
crew = pd.read_csv('processed/title_rating_crew.csv')
principal = pd.read_csv('processed/title_rating_principal.csv')

In [16]:
crew['directors'].replace({'\\N': ''}, inplace = True)
crew['writers'].replace({'\\N': ''}, inplace = True)
principal['nconst'].replace({'\\N': ''}, inplace = True)

In [17]:
names = pd.read_csv('processed/name_basics.csv')

In [18]:
crew

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,directors,writers
0,tt0000502,movie,Bohemios,Bohemios,0,1970-01-01 00:00:00.000001905,NaN,100.0,NaN,4.5,14,nm0063413,"nm0063413,nm0657268,nm0675388"
1,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1970-01-01 00:00:00.000001906,NaN,70.0,"Action,Adventure,Biography",6.0,754,nm0846879,nm0846879
2,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1970-01-01 00:00:00.000001907,NaN,90.0,Drama,4.6,17,nm0141150,nm0141150
3,tt0000615,movie,Robbery Under Arms,Robbery Under Arms,0,1970-01-01 00:00:00.000001907,NaN,NaN,Drama,4.5,23,nm0533958,"nm0092809,nm0533958"
4,tt0000630,movie,Hamlet,Amleto,0,1970-01-01 00:00:00.000001908,NaN,NaN,Drama,3.8,24,nm0143333,nm0000636
...,...,...,...,...,...,...,...,...,...,...,...,...,...
323829,tt9916362,movie,Coven,Akelarre,0,2020-01-01,NaN,92.0,"Drama,History",6.4,4447,nm1893148,"nm1893148,nm3471432"
323830,tt9916428,movie,The Secret of China,Hong xing zhao yao Zhong guo,0,2019-01-01,NaN,NaN,"Adventure,History,War",3.8,14,nm0910951,
323831,tt9916460,tvMovie,Pink Taxi,Pink Taxi,0,2019-01-01,NaN,NaN,Comedy,9.3,17,nm7048843,"nm7048843,nm8691452"
323832,tt9916538,movie,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,0,2019-01-01,NaN,123.0,Drama,8.3,6,nm4457074,"nm4843252,nm4900525,nm2679404"


In [19]:
principal

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,ordering,nconst,category,job,characters
0,tt0000502,movie,Bohemios,Bohemios,0,1970-01-01 00:00:00.000001905,NaN,100.0,NaN,4.5,14,1,nm0215752,actor,\N,\N
1,tt0000502,movie,Bohemios,Bohemios,0,1970-01-01 00:00:00.000001905,NaN,100.0,NaN,4.5,14,2,nm0252720,actor,\N,\N
2,tt0000502,movie,Bohemios,Bohemios,0,1970-01-01 00:00:00.000001905,NaN,100.0,NaN,4.5,14,3,nm0063413,director,\N,\N
3,tt0000502,movie,Bohemios,Bohemios,0,1970-01-01 00:00:00.000001905,NaN,100.0,NaN,4.5,14,4,nm0657268,writer,\N,\N
4,tt0000502,movie,Bohemios,Bohemios,0,1970-01-01 00:00:00.000001905,NaN,100.0,NaN,4.5,14,5,nm0675388,writer,\N,\N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2885442,tt9916730,movie,6 Gunn,6 Gunn,0,2017-01-01,NaN,116.0,NaN,8.4,5,5,nm10538612,director,\N,\N
2885443,tt9916730,movie,6 Gunn,6 Gunn,0,2017-01-01,NaN,116.0,NaN,8.4,5,6,nm10538614,producer,producer,\N
2885444,tt9916730,movie,6 Gunn,6 Gunn,0,2017-01-01,NaN,116.0,NaN,8.4,5,7,nm10538613,producer,associate producer,\N
2885445,tt9916730,movie,6 Gunn,6 Gunn,0,2017-01-01,NaN,116.0,NaN,8.4,5,8,nm1957275,cinematographer,\N,\N


In [20]:
#creating some useful data stuctures for further processing
title_genre = title_rating[['tconst', 'genres']]

In [21]:
title_genre['genres'] = title_genre['genres'].apply(lambda x: split_names(x))

/home/tanmay/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
title_genre_dict = title_genre.set_index('tconst').to_dict()['genres']

In [23]:
names['knownForTitles'].fillna('', inplace = True)

In [24]:
names = names[['nconst', 'knownForTitles']]

In [25]:
names['knownForTitles'] = names['knownForTitles'].apply(lambda x: split_names(x))


In [28]:
names

,nconst,knownForTitles
0,nm0000001,"[tt0050419, tt0031983, tt0072308, tt0053137]"
1,nm0000002,"[tt0037382, tt0038355, tt0071877, tt0117057]"
2,nm0000003,"[tt0057345, tt0056404, tt0054452, tt0049189]"
3,nm0000004,"[tt0080455, tt0078723, tt0072562, tt0077975]"
4,nm0000005,"[tt0050986, tt0083922, tt0060827, tt0069467]"
...,...,...
964582,nm9993616,[tt4844148]
964583,nm9993650,[tt8739208]
964584,nm9993690,[tt7888884]
964585,nm9993691,[tt7888884]


In [27]:
names['nconst'].dropna(inplace = True) 

In [29]:
names_title_dict = names.set_index('nconst').to_dict()['knownForTitles']

In [39]:
t = 'tt0000502'

In [51]:
people = set()

In [52]:
people.update(principal[principal['tconst'] == t]['nconst'].values)

In [54]:
people.update(split_names(crew[crew['tconst'] == t]['directors'].values[0]))
people.update(split_names(crew[crew['tconst'] == t]['writers'].values[0]))

In [55]:
people

{'nm0063413', 'nm0215752', 'nm0252720', 'nm0657268', 'nm0675388'}

In [67]:
related = set()
for p in people:
    print(names_title_dict[p])
    related.update(names_title_dict[p])

['tt0004406', 'tt0000941', 'tt0002886', 'tt0024495']
['tt0000502']
['tt0031112', 'tt0208070', 'tt0088954', 'tt0064102']
['tt0088954', 'tt0064102', 'tt0208070', 'tt0031112']
['tt0000502']


In [75]:
genres = []
for r in related:
    try:
        genres = genres+title_genre_dict[r]
    except:
        pass

In [77]:
genres

['Comedy', 'Musical', 'Drama', 'Comedy', 'Musical']

In [78]:
c = Counter(genres)

In [83]:
c.most_common(2)

[('Comedy', 2), ('Musical', 2)]

In [66]:
def impute_genre(title, names_title_dict, 
                 title_genre_dict, 
                 principal,
                 crew):
    people = set()
    
    people.update(principal[principal['tconst'] == title]['nconst'].values)
    people.update(split_names(crew[crew['tconst'] == title]['directors'].values[0]))
    people.update(split_names(crew[crew['tconst'] == title]['writers'].values[0]))
    
    related = set()
#     print(people)
    for p in people:
#         print(names_title_dict[p])
        try:
            related.update(names_title_dict[p])
        except:
            pass
    
    genres = []
    for r in related:
        try:
            genres = genres+title_genre_dict[r]
        except:
            pass
        
    c = Counter(genres)
    comm = c.most_common(2)
#     print(comm)
    impute = []
    for item in comm:
        impute.append(item[0])
        
    return impute
    

In [67]:
impute_genre('tt0076430', names_title_dict, 
                 title_genre_dict, 
                 principal,
                 crew)

['Drama', 'Comedy']

In [64]:
# missing_genres =  title_rating[title_rating['genres'] == '']['tconst'].values

In [72]:
for key in tqdm(title_genre_dict.keys(), position=0, leave=True):
    if title_genre_dict[key] == []:
        title_genre_dict[key] = impute_genre(key, names_title_dict, 
                                             title_genre_dict, 
                                             principal,
                                             crew)
        

100%|██████████| 323834/323834 [22:46<00:00, 382.43it/s] 

                                                  
100%|██████████| 323834/323834 [22:46<00:00, 237.03it/s]

In [76]:
e_c = 0
for k in list(title_genre_dict.keys()):
    if title_genre_dict[k] == []:
        e_c+=1

In [81]:
with open('processed/title_genre_dict.json', 'w') as f:
    json.dump(title_genre_dict, f)

In [84]:

for k in tqdm(list(title_genre_dict.keys()),  position=0, leave=True):
    title_genre_dict[k] = ','.join(title_genre_dict[k])

100%|██████████| 323834/323834 [00:00<00:00, 1790366.87it/s]

In [86]:
t_g_df = pd.DataFrame.from_dict(title_genre_dict, orient='index')

In [88]:
title_genre_new = pd.merge(title_genre, t_g_df, left_on='tconst', right_on=t_g_df)

In [89]:
title_genre_new

,tconst,genres,0
0,tt0000502,[],"Comedy,Musical"
1,tt0000574,"[Action, Adventure, Biography]","Action,Adventure,Biography"
2,tt0000591,[Drama],Drama
3,tt0000615,[Drama],Drama
4,tt0000630,[Drama],Drama
...,...,...,...
323829,tt9916362,"[Drama, History]","Drama,History"
323830,tt9916428,"[Adventure, History, War]","Adventure,History,War"
323831,tt9916460,[Comedy],Comedy
323832,tt9916538,[Drama],Drama


In [99]:
title_rating_new = pd.merge(title_rating, title_genre_new, left_on='tconst', right_on = 'tconst')

In [106]:
title_rating_new.drop(columns=['genres_y', 'genres_x'], inplace=True)

In [109]:
title_rating_new[title_rating_new[0] == '']

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,averageRating,numVotes,0
13,tt0001010,movie,Protección de un convoy de víveres en el puent...,Protección de un convoy de víveres en el puent...,0,1970-01-01 00:00:00.000001909,NaN,NaN,4.6,14,
15,tt0001038,movie,Sherlock Holmes VI,Sherlock Holmes VI,0,1970-01-01 00:00:00.000001910,NaN,NaN,3.8,21,
19,tt0001113,movie,Amor gitano,Amor gitano,0,1970-01-01 00:00:00.000001910,NaN,NaN,5.1,16,
277,tt0005040,movie,Butter,Butter,0,1970-01-01 00:00:00.000001916,NaN,NaN,6.7,11,
366,tt0005869,movie,Pasionaria,Pasionaria,0,1970-01-01 00:00:00.000001915,NaN,NaN,4.4,11,
...,...,...,...,...,...,...,...,...,...,...,...
318737,tt9013770,movie,The Witches of Gambaga,The Witches of Gambaga,0,2011-01-01,NaN,55.0,7.0,9,
318887,tt9031770,movie,Ogar: Will of Steel,Ogar: Will of Steel,0,2017-01-01,NaN,82.0,5.2,5,
319534,tt9114062,tvMovie,Family Classics: Scrooge (1951) II,Family Classics: Scrooge (1951) II,0,2018-01-01,NaN,NaN,7.4,11,
322768,tt9723258,movie,Little Wound's Warriors,Little Wound's Warriors,0,2017-01-01,NaN,57.0,6.6,12,


In [113]:
title_rating_new.rename(columns={0:'genres'}, inplace = True)

In [115]:
title_rating_new.to_csv('processed/title_genre_new.csv', index = False)

In [116]:
title_rating_new.isnull().sum()

tconst                 0
titleType              0
primaryTitle           0
originalTitle          0
isAdult                0
startYear             34
endYear           323834
runtimeMinutes     35737
averageRating          0
numVotes               0
genres                 0
dtype: int64

In [16]:
title_rating_new = pd.read_csv('processed/title_genre_new.csv')

Imputing start date

Imputing the **Start Date**

We can use the TMDB API to impute these values

In [17]:
def get_release_date(name):
#     print(name)
    try:
        movie = Movie()
        res = movie.search(name)[0]['release_date']
        print(pd.to_datetime(res))
        return pd.to_datetime(res)
    except:
        return pd.to_datetime('1980')
    

In [18]:
title_rating_new['startYear'] = title_rating_new.progress_apply(lambda x: get_release_date(x['primaryTitle']) if pd.isna(x['startYear']) else x['startYear'], axis = 1)

 15%|█▍        | 47781/323834 [00:07<17:03, 269.71it/s] 

1976-05-31 00:00:00


 17%|█▋        | 56231/323834 [00:07<08:14, 540.91it/s]

1987-06-01 00:00:00
2012-10-28 00:00:00


 42%|████▏     | 135216/323834 [00:09<00:05, 31766.98it/s]

...

2021-01-15 00:00:00
2017-07-17 00:00:00


100%|█████████▉| 322361/323834 [00:16<00:00, 16231.20it/s]

2016-04-14 00:00:00


100%|██████████| 323834/323834 [00:17<00:00, 18817.21it/s]

2013-05-21 00:00:00


In [19]:
title_rating_new.isnull().sum()

tconst                 0
titleType              0
primaryTitle           0
originalTitle          0
isAdult                0
startYear              0
endYear           323834
runtimeMinutes     35737
averageRating          0
numVotes               0
genres               540
dtype: int64

In [20]:
title_rating_new.drop(columns=['endYear'], inplace = True)

In [24]:
no_runtime = title_rating_new[title_rating_new['runtimeMinutes'].isna()]['primaryTitle'].values

In [22]:
title_rating_new['runtimeMinutes'].mean()

92.72670663005863

In [26]:
def get_runtime(name):
    
    try:
        movie = Movie()
        res = movie.search(name)[0]
        movie_id = res['id']
        
        url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={API_KEY}"
        
        res = requests.get(url).json()
        return res['runtime']
    except:
        #returing mean runtime ignoring nulls
        return 93
        

In [27]:
runtime_dict = {}
for item in tqdm(no_runtime, position=0, leave=True):
    runtime_dict[item] = get_runtime(item)

100%|██████████| 35737/35737 [2:22:04<00:00,  4.19it/s]   


In [28]:
with open('runtime_impute_dict.json', 'w') as f:
    json.dump(runtime_dict, f)


In [35]:
title_rating_new['runtimeMinutes'] = title_rating_new.progress_apply(lambda x: runtime_dict[x['primaryTitle']] if pd.isna(x['runtimeMinutes']) else x['runtimeMinutes'], axis = 1)




  0%|          | 0/323834 [00:00<?, ?it/s]


  0%|          | 82/323834 [00:00<06:34, 819.95it/s]


  1%|▏         | 4406/323834 [00:00<04:34, 1161.91it/s]


  2%|▏         | 7482/323834 [00:00<03:13, 1633.43it/s]


  3%|▎         | 10032/323834 [00:00<02:18, 2271.12it/s]


  4%|▍         | 13560/323834 [00:00<01:38, 3157.35it/s]


  6%|▌         | 17830/323834 [00:00<01:09, 4371.95it/s]


  7%|▋         | 22091/323834 [00:00<00:50, 5982.55it/s]


  8%|▊         | 25914/323834 [00:00<00:37, 8009.28it/s]


  9%|▉         | 30127/323834 [00:00<00:27, 10579.79it/s]


 11%|█         | 34083/323834 [00:01<00:21, 13559.51it/s]


 12%|█▏        | 37951/323834 [00:01<00:16, 16840.08it/s]


 13%|█▎        | 41936/323834 [00:01<00:13, 20368.21it/s]


 14%|█▍        | 45805/323834 [00:01<00:11, 23606.10it/s]


 15%|█▌        | 49682/323834 [00:01<00:10, 26743.39it/s]


 17%|█▋        | 53529/323834 [00:01<00:09, 28776.34it/s]


 18%|█▊        | 57272/323834 [00:01<00:08, 30788.39it/s]


 19%|█

In [30]:
len(runtime_dict.keys())

34938

In [36]:
title_rating_new.isnull().sum()

tconst               0
titleType            0
primaryTitle         0
originalTitle        0
isAdult              0
startYear            0
runtimeMinutes    1730
averageRating        0
numVotes             0
genres             540
dtype: int64

We fill up the missed values with the mean runtime of 93 mins

In [37]:
title_rating_new['runtimeMinutes'].fillna(93, inplace = True)

In [38]:
title_rating_new.to_csv('processed/title_rating_new.csv', index = False)

## Feature Engineering

I'll Use this section to explore some feature engineering

### Professional Quality

A strong intuitive predictor of the quality of a movie comes from the cast and the crew of that movie. Thus we need to figure out a way to encode these values